In [5]:
import os
import pandas as pd 
from bs4 import BeautifulSoup


In [6]:
SCORE_DIR = 'data/scores'

In [7]:
box_scores = os.listdir(SCORE_DIR)

In [9]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

8920